In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os 
import random
import plotly.express as px

In [2]:
df = pd.read_parquet("Shopping Carts.parquet")
df.reset_index(inplace=True)

### Prüfen Sie, ob es beim Produkt 13176 auffällige Muster beim Wiederbestellverhalten gibt. 

In [3]:
df.groupby("order_id").agg(order_size=("add_to_cart_order","max")).reset_index()
df = pd.merge(df,df.groupby("order_id").agg(order_size=("add_to_cart_order","max")).reset_index())

In [4]:
df_auf1 = df[df.product_id == 13176]

In [5]:
liste = df_auf1.groupby("user_id").agg(order_id=("order_id","min")).reset_index().order_id.to_list()
df_auf1.loc[df_auf1.order_id.isin(liste),"reordered"] = 0

In [6]:
user = df_auf1.user_id.unique().tolist()
banana_order = df_auf1.order_id.unique().tolist()
df_user = df[df.user_id.isin(user)]

In [7]:
df_user["Banana"] = False
df_user.loc[df_user.order_id.isin(banana_order),"Banana"] = True

C:\Users\paulh\AppData\Local\Temp\ipykernel_13860\93905981.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user["Banana"] = False


In [8]:
df_idk = df_user.groupby(["user_id","order_id"]).agg(Banana=("Banana","max"),days_since_prior_order = ("days_since_prior_order","mean")).reset_index()
df_idk["Summe"] = df_idk.days_since_prior_order

In [9]:
df_idk["Cum"] = df_idk.Summe.cumsum()

In [10]:
df_idk2 = df_idk.loc[df_idk.Banana == True]

In [11]:
df_idk2["Cumsum"] = df_idk2["Cum"].shift()

C:\Users\paulh\AppData\Local\Temp\ipykernel_13860\1643783414.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_idk2["Cumsum"] = df_idk2["Cum"].shift()


In [12]:
df_idk2["Days"] = df_idk2.Cum - df_idk2.Cumsum

C:\Users\paulh\AppData\Local\Temp\ipykernel_13860\3771161101.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_idk2["Days"] = df_idk2.Cum - df_idk2.Cumsum


In [14]:
df_idk2["user2"] = df_idk2.user_id.shift()

C:\Users\paulh\AppData\Local\Temp\ipykernel_13860\25003005.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_idk2["user2"] = df_idk2.user_id.shift()


In [16]:
df_idk2.loc[df_idk2.user_id != df_idk2.user2,"Days"] = None

In [17]:
df_idk2

,user_id,order_id,Banana,days_since_prior_order,Summe,Cum,Cumsum,Days,user2
18,7,553865,True,2.0,2.0,199.0,NaN,NaN,NaN
21,40,74936,True,7.0,7.0,216.0,199.0,NaN,7.0
22,40,200867,True,20.0,20.0,236.0,216.0,20.0,40.0
23,40,203217,True,14.0,14.0,250.0,236.0,14.0,40.0
25,40,393331,True,21.0,21.0,283.0,250.0,33.0,40.0
...,...,...,...,...,...,...,...,...,...
261994,206162,34456,True,12.0,12.0,2378244.0,2378175.0,NaN,206153.0
262006,206162,424565,True,27.0,27.0,2378470.0,2378244.0,226.0,206162.0
262011,206168,109567,True,6.0,6.0,2378507.0,2378470.0,NaN,206162.0
262017,206168,465130,True,9.0,9.0,2378528.0,2378507.0,21.0,206168.0
